Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.56.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacity_1
cloud_shell
northcentralus
d2d90bd8-e567-4097-88c9-9532cc375686


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,udacity_1,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [8151c0c8][b6d044b9-d985-443c-a8cb-76a72cd6808e], (This step will run and generate new outputs)
Submitted PipelineRun f36e6323-c0f6-4f3a-82fb-374f3ddcd532
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f36e6323-c0f6-4f3a-82fb-374f3ddcd532?wsid=/subscriptions/d2d90bd8-e567-4097-88c9-9532cc375686/resourcegroups/cloud_shell/workspaces/udacity_1&tid=f3822f31-4d32-4719-a061-c45fac0a64ab


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

2024-09-02 21:44:48.354016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38/lib/python3.9/site-packages/cv2/../../lib64:
2024-09-02 21:44:48.354088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


In [14]:
pipeline_run.wait_for_completion()

PipelineRunId: ce41bd7e-867b-46c4-baaa-cddfaefc6dd7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ce41bd7e-867b-46c4-baaa-cddfaefc6dd7?wsid=/subscriptions/d2d90bd8-e567-4097-88c9-9532cc375686/resourcegroups/cloud_shell/workspaces/udacity_1&tid=f3822f31-4d32-4719-a061-c45fac0a64ab
PipelineRun Status: Running


StepRunId: 86190a49-3c38-43a5-9a04-a3c66409f30c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/86190a49-3c38-43a5-9a04-a3c66409f30c?wsid=/subscriptions/d2d90bd8-e567-4097-88c9-9532cc375686/resourcegroups/cloud_shell/workspaces/udacity_1&tid=f3822f31-4d32-4719-a061-c45fac0a64ab
StepRun( automl_module ) Status: Running


ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/ba69465a-d7f1-4111-adf3-ef1431c2666c/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,ba69465a-d7f1-4111-adf3-ef1431c2666c_37,ba69465a-d7f1-4111-adf3-ef1431c2666c_1,ba69465a-d7f1-4111-adf3-ef1431c2666c_36,ba69465a-d7f1-4111-adf3-ef1431c2666c_26,ba69465a-d7f1-4111-adf3-ef1431c2666c_12,ba69465a-d7f1-4111-adf3-ef1431c2666c_9,ba69465a-d7f1-4111-adf3-ef1431c2666c_8,ba69465a-d7f1-4111-adf3-ef1431c2666c_13,ba69465a-d7f1-4111-adf3-ef1431c2666c_10,ba69465a-d7f1-4111-adf3-ef1431c2666c_15,...,ba69465a-d7f1-4111-adf3-ef1431c2666c_30,ba69465a-d7f1-4111-adf3-ef1431c2666c_41,ba69465a-d7f1-4111-adf3-ef1431c2666c_40,ba69465a-d7f1-4111-adf3-ef1431c2666c_33,ba69465a-d7f1-4111-adf3-ef1431c2666c_49,ba69465a-d7f1-4111-adf3-ef1431c2666c_3,ba69465a-d7f1-4111-adf3-ef1431c2666c_17,ba69465a-d7f1-4111-adf3-ef1431c2666c_42,ba69465a-d7f1-4111-adf3-ef1431c2666c_50,ba69465a-d7f1-4111-adf3-ef1431c2666c_44
AUC_micro,[0.9797451880234226],[0.9783641467160662],[0.9696915130986619],[0.9780501564655144],[0.9786896502494928],[0.8475889113269979],[0.9790036405000448],[0.9557049007439882],[0.9741933909150988],[0.9775185191155036],...,[0.9783447122945742],[0.9781238414759107],[0.9793676444514035],[0.9770006055986793],[0.9805473414678514],[0.9664817940457906],[0.8934985412670597],[0.9773522673107966],[0.9803983135343246],[0.9781102097489875]
average_precision_score_weighted,[0.9541358029182306],[0.950146970738567],[0.9382661445555986],[0.9497977778114575],[0.9523114199391449],[0.9258954734955656],[0.9529980385601647],[0.9152646769687088],[0.9437518530646575],[0.9536793029588512],...,[0.9526950938003463],[0.9509562036584922],[0.953909745885795],[0.9482474801208366],[0.9561773847166365],[0.9309800314335599],[0.939337955537193],[0.9481326502168407],[0.9554219461872495],[0.9514575108916562]
accuracy,[0.9098634294385433],[0.9116843702579667],[0.9004552352048558],[0.9101669195751139],[0.9138088012139606],[0.7405159332321699],[0.9104704097116844],[0.8880121396054628],[0.9083459787556905],[0.9068285280728376],...,[0.9059180576631259],[0.9092564491654022],[0.9119878603945372],[0.9059180576631259],[0.9156297420333839],[0.8916540212443096],[0.8006069802731411],[0.9080424886191198],[0.9132018209408195],[0.9059180576631259]
precision_score_micro,[0.9098634294385433],[0.9116843702579667],[0.9004552352048558],[0.9101669195751139],[0.9138088012139606],[0.7405159332321699],[0.9104704097116844],[0.8880121396054628],[0.9083459787556905],[0.9068285280728376],...,[0.9059180576631259],[0.9092564491654022],[0.9119878603945372],[0.9059180576631259],[0.9156297420333839],[0.8916540212443096],[0.8006069802731411],[0.9080424886191198],[0.9132018209408195],[0.9059180576631259]
f1_score_micro,[0.9098634294385433],[0.9116843702579667],[0.9004552352048558],[0.9101669195751139],[0.9138088012139606],[0.7405159332321699],[0.9104704097116844],[0.8880121396054628],[0.9083459787556905],[0.9068285280728376],...,[0.9059180576631259],[0.9092564491654022],[0.9119878603945372],[0.9059180576631259],[0.9156297420333839],[0.8916540212443096],[0.8006069802731411],[0.9080424886191198],[0.9132018209408195],[0.9059180576631259]
average_precision_score_macro,[0.8195506429132136],[0.8028233128807174],[0.7619834352475827],[0.8028257572649862],[0.8136182410340502],[0.7333442342130015],[0.8161730247772492],[0.7088516510499497],[0.7841343649033216],[0.8180629123186702],...,[0.8144008894526185],[0.8081696387188946],[0.8190021212604568],[0.7973949197161567],[0.8274631628931424],[0.745700854487855],[0.7692719309990121],[0.7957729533606923],[0.8245332846993769],[0.8106918416443103]
norm_macro_recall,[0.4935231648967209],[0.4955737458946703],[0.33372788956871124],[0.4464996563841237],[0.464810399983699],[0.492280220136446],[0.44210489268255637],[0.0],[0.3307724225567614],[0.22012347943028288],...,[0.2759365153460147],[0.4288965206808597],[0.5077568273973923],[0.42513712218461897],[0.43607355417368265],[0.037256852404477625],[0.6854673639012536],[0.39911030347487353],[0.47596819098744647],[0.44408323098952107]
AUC_weighted,[0.9449992312636728],[0.942328104073932],[0.9216222374117111],[0.9394485845063509],[0.940516

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/ba69465a-d7f1-4111-adf3-ef1431c2666c/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

2024-09-02 22:17:50.132024: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-09-02 22:17:57.291143: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-02 22:17:57.291292: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (monaejam2): /proc/driver/nvidia/version does not exist


PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/monaejam2/code/Users/monaejam')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('3...nit_type': 'cpu'}), reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))]))], flatten_transform=False, weights=[0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.25, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [18]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.4, gamma=5, max_depth=6, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.7708333333333335, reg_lambda=1.5625, subsample=0.5, tree_method='auto'))], verbose=False)), ('37', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')),

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [ ]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [44]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,cf83288b-f7e6-4990-b8be-36c0a401b36a,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [26]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [24]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [27]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  27a9939e-ad98-4dd2-a1dd-5541c0c3fc87


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [53]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [54]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


In [37]:
remote_run = experiment.submit(automl_config, show_output= True)
remote_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml


Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_09736824-b72f-4a9f-a2d5-532eee62be40,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, your input data has been split into a training dataset and a holdout validation dataset.
      

In [ ]:
best_run, fitted_model = remote_run.get_output()


In [ ]:
best_run


In [ ]:
best_run.register_model(model_name='best_run_automl', model_path='./outputs/')


In [42]:
from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

In [52]:
model = remote_run.register_model(model_name = 'best_run_automl.pkl')

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_2_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservices", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-09-03 02:53:08+00:00 Creating Container Registry if not exists.
2024-09-03 02:53:09+00:00 Use the existing image.
2024-09-03 02:53:10+00:00 Generating deployment configuration.
2024-09-03 02:53:12+00:00 Submitting deployment to compute..
2024-09-03 02:53:20+00:00 Checking the status of deployment aciservices..
2024-09-03 02:56:57+00:00 Checking the status of inference endpoint aciservices.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [48]:
%run endpoint.py


Response from urllib request:
{"Results": ["no", "no"]}
Response from requests:
{'Results': ['no', 'no']}


In [50]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)


2024-09-03T01:02:44,264405792+00:00 - rsyslog/run 
2024-09-03T01:02:44,269045245+00:00 - gunicorn/run 
2024-09-03T01:02:44,273232937+00:00 | gunicorn/run | 
2024-09-03T01:02:44,275265753+00:00 | gunicorn/run | ###############################################
2024-09-03T01:02:44,276986729+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-09-03T01:02:44,280194589+00:00 | gunicorn/run | ###############################################
2024-09-03T01:02:44,281285726+00:00 | gunicorn/run | 
2024-09-03T01:02:44,283092811+00:00 - nginx/run 
2024-09-03T01:02:44,288257246+00:00 | gunicorn/run | 
2024-09-03T01:02:44,291294691+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240709.v1
2024-09-03T01:02:44,293011033+00:00 | gunicorn/run | 
2024-09-03T01:02:44,298973454+00:00 | gunicorn/run | 
2024-09-03T01:02:44,301742174+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [49]:
from azureml.core import Workspace, Webservice

# Load the workspace from the config file
ws = Workspace.from_config()

# List all deployed web services in the workspace
services = Webservice.list(ws)

# Print details of each service
for service in services:
    print(f"Name: {service.name}")
    print(f"Scoring URI: {service.scoring_uri}")
    print(f"Region: {service.location}")
    print(f"State: {service.state}")
    print("-" * 40)


Name: aciservice
Scoring URI: http://1b069481-9607-4f37-b5c7-5fab51c30dac.northcentralus.azurecontainer.io/score
Region: northcentralus
State: None
----------------------------------------
